In [1]:
import numpy as np
import pandas as pd
import random
import os
import csv
import torch
import torchvision.models as models
import torchvision.transforms as transforms

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from PIL import Image
from sklearn import preprocessing

C:\Users\busr4\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 'Could not find module 'C:\Users\busr4\AppData\Local\Programs\Python\Python310\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.RandomRotation(20),      # Rotate the image by up to 20 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),  # Randomly change brightness, contrast, and saturation
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.Resize((224, 224))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.Resize((224, 224))
])


class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = torch.from_numpy(labels).type(torch.LongTensor)
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        label = self.labels[index]
        image = Image.open(image_path)

        if self.transform:
            image = self.transform(image)

        return image, label


In [3]:
label_encoder = preprocessing.LabelEncoder()

# Load pre-trained models
resnet = models.resnet50(pretrained=True)

toy_data_path = 'toy_dataset_label.csv'
toy_data = pd.read_csv(toy_data_path, on_bad_lines='skip', delimiter='\t',quoting=csv.QUOTE_NONE)
labels_toy_data = toy_data.iloc[:, 9].values

classes = set()
for val in labels_toy_data:
    if pd.notna(val):
        classes.add(val)
num_classes = len(classes)
resnet.fc = torch.nn.Linear(resnet.fc.in_features, num_classes)

C:\Users\busr4\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\busr4\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
def train_model(model, train_loader, val_loader, num_epochs=10):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = torch.nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        model.train()
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # Validation step
        model.eval()
        with torch.no_grad():
            # Implement validation step
            pass

    # Return the trained model
    return model

all_image_paths = []
for i in range(10):
    img_path = 'D:/Dersler-20210512T064119Z-001/Dersler/2023_Fall/COMP411/COMP411-FineART/toy_dataset/toy_dataset/{}.jpg'.format(i+1)
    all_image_paths.append(img_path)
    
all_labels = labels_toy_data[:10]
train_paths, test_paths, train_labels, test_labels = train_test_split(all_image_paths, all_labels, test_size=0.2, random_state=42)

# Further split the training set into training and validation
train_paths, val_paths, train_labels, val_labels = train_test_split(
    train_paths, train_labels, test_size=0.25, random_state=42)

train_dataset = CustomDataset(train_paths, label_encoder.fit_transform(train_labels), transform=train_transform)
val_dataset = CustomDataset(val_paths, label_encoder.fit_transform(val_labels), transform=test_transform)
test_dataset = CustomDataset(test_paths, label_encoder.fit_transform(test_labels), transform=test_transform)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

trained_resnet = train_model(resnet, train_loader, val_loader, num_epochs=10)


C:\Users\busr4\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [5]:
def test_model(model, test_loader):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = torch.nn.CrossEntropyLoss()
    
    model.eval()
    with torch.no_grad():
        total = 0
        correct = 0
        test_loss = 0
        criterion = torch.nn.CrossEntropyLoss()  # Replace with your loss function

        for images, labels in test_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()


        average_loss = test_loss / len(test_loader)
        accuracy = correct / total

        print(f'Test Loss: {average_loss:.4f}, Test Accuracy: {accuracy:.4f}')

In [6]:
test_model(trained_resnet, test_loader)

Test Loss: 11.4738, Test Accuracy: 0.5000
